In [3]:
from google.cloud import bigquery
import gc
import pandas as pd
import numpy as np
import datetime
import os
import json
import glob
from haversine import haversine
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/docs/merkle-gdyr-prod-9783607c77a1.json"

zip_centers=json.load(open("/mnt/clients/juba/hqjubaapp02/sharefolder/Docs/Geo_mapping/updated_zip_centers_JL_2019-05-23.json"))
client = bigquery.Client()

print(datetime.datetime.now())
print(os.getcwd())

2020-12-08 20:42:34.266363
/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/jian/Post_Campaign_Analysis


# Audience Report

In [6]:
df_31_green=pd.read_excel("/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/jian/Audience_selections/GY_green_store_matching_pair_BenJL_20201103.xlsx",sheet_name="Sheet1",dtype=str)
df_9_single=pd.read_excel("/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/jian/Audience_selections/GY_twelve_matching_pair_BenJL_20201103.xlsx",sheet_name="Sheet1",dtype=str)
df_31_green=df_31_green[df_31_green['final_selection']=="1"]
df_9_single=df_9_single[df_9_single['final_selection']=="1"]

# 1
list_test_store_str_id_part_1=[str(int(x)).zfill(4) for x in df_31_green['green_store_id'].tolist()]
list_control_store_str_id_part_1=[str(int(x)).zfill(4) for x in df_31_green['matching_store_id'].tolist()]
# 2
list_test_store_str_id_part_2=[str(int(x)).zfill(4) for x in df_9_single['twelve_store_id'].tolist()]
list_control_store_str_id_part_2=[str(int(x)).zfill(4) for x in df_9_single['matching_store_id'].tolist()]

print(len(list_test_store_str_id_part_1),len(list_control_store_str_id_part_1))
print(len(list_test_store_str_id_part_2),len(list_control_store_str_id_part_2))

print(len(set(list_test_store_str_id_part_1+list_control_store_str_id_part_1+list_test_store_str_id_part_2+list_control_store_str_id_part_2)))

31 31
9 9
80


In [8]:
list_40_test_stores=list_test_store_str_id_part_1+list_test_store_str_id_part_2
list_40_control_stores=list_control_store_str_id_part_1+list_control_store_str_id_part_2
print(len(list_40_test_stores),len(list_40_test_stores))

40 40


In [4]:
print(datetime.datetime.now())
query_crm="""
select t1.indiv_key, t1.best_email_txt, t1.best_phone_num, t2.postal_cd, t2.state_province_cd, t2.country_cd from mdb.indiv_summary as t1
join mdb.addr as t2 on
t1.cr_addr_key=t2.cr_addr_key
"""

print(query_crm)

df_all_ids=client.query(query_crm).result().to_dataframe()
print(datetime.datetime.now())
print(df_all_ids.shape,df_all_ids['indiv_key'].nunique())

2020-12-08 20:43:25.075206

select t1.indiv_key, t1.best_email_txt, t1.best_phone_num, t2.postal_cd, t2.state_province_cd, t2.country_cd from mdb.indiv_summary as t1
join mdb.addr as t2 on
t1.cr_addr_key=t2.cr_addr_key



/home/jliang/.local/lib/python3.7/site-packages/google/cloud/bigquery/client.py:441: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


2020-12-08 20:50:33.900055
(7591179, 6) 7591179


In [10]:
str_start_date="'2020-11-16'"
str_end_date="'2020-12-02'"

In [22]:
# Service & Tire & null
print(datetime.datetime.now())
query_both_by_id="""
            select indiv_key, min(invoice_date) as min_date, max(invoice_date) as max_date, 
            count(distinct(case when store_id='ECOM' then trn_id end)) as Ecomm_Trans,
            sum(case when store_id='ECOM' then line_item_total_amt else 0 end) as Ecomm_Revenue,
            count(distinct(case when store_id!='ECOM' then trn_id end)) as Store_Trans,
            sum(case when store_id!='ECOM' then line_item_total_amt else 0 end) as Store_Revenue,
            count(distinct trn_id) as Total_Trans,
            sum(line_item_total_amt) as Total_Revenue
            from mdb.trn_detail
            where indiv_key is not null and invoice_date between %s and %s
            group by indiv_key
            """  %(str_start_date,str_end_date)
print(query_both_by_id)
df_both_by_id=client.query(query_both_by_id).result().to_dataframe()
print(df_both_by_id.shape,df_both_by_id['indiv_key'].nunique())
print(datetime.datetime.now())

2020-12-09 00:13:38.151693

            select indiv_key, min(invoice_date) as min_date, max(invoice_date) as max_date, 
            count(distinct(case when store_id='ECOM' then trn_id end)) as Ecomm_Trans,
            sum(case when store_id='ECOM' then line_item_total_amt else 0 end) as Ecomm_Revenue,
            count(distinct(case when store_id!='ECOM' then trn_id end)) as Store_Trans,
            sum(case when store_id!='ECOM' then line_item_total_amt else 0 end) as Store_Revenue,
            count(distinct trn_id) as Total_Trans,
            sum(line_item_total_amt) as Total_Revenue
            from mdb.trn_detail
            where indiv_key is not null and invoice_date between '2020-11-25' and '2020-12-02'
            group by indiv_key
            


/home/jliang/.local/lib/python3.7/site-packages/google/cloud/bigquery/client.py:441: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


(53661, 9) 53661
2020-12-09 00:13:45.172595


In [38]:
final_uploaded_folder="/mnt/clients/juba/hqjubaapp02/sharefolder/GoodYear/jian/Audience_selections/output_cleaned_audience_2020-11-06/"
list_uploaded_main_aud_files=glob.glob(final_uploaded_folder+"*.csv")

list_audience_national=[x for x in list_uploaded_main_aud_files if "balance" in x.lower() and "tireandservice" in x.lower()]
list_audience_national.sort()
# list_audience_national

list_audience_40_stores=[x for x in list_uploaded_main_aud_files if "_stores_" in x.lower() and "tireandservice" in x.lower() and "_ecom" in x.lower()]
list_audience_40_stores.sort()
# list_audience_40_stores

In [47]:
df_id_segment_label=pd.DataFrame()

for file in list_audience_national+list_audience_40_stores:
    df=pd.read_csv(file,usecols=['indiv_key'])
    df['segment']=os.path.basename(file)
    df_id_segment_label=df_id_segment_label.append(df)
print(df_id_segment_label.shape,df_id_segment_label['indiv_key'].nunique())

(284729, 2) 284489


In [49]:
df=df_id_segment_label.groupby('indiv_key')['segment'].count().to_frame().reset_index()
df=df.sort_values("segment",ascending=False)
df_dup=df[df['segment']>1]
df_dup.shape

(240, 2)

In [58]:
for i in range(len(df_dup)):
    df=df_id_segment_label[df_id_segment_label['indiv_key']==df_dup['indiv_key'].tolist()[i]]
    df=df[df['segment'].str.contains("Test")]
    if len(df)>0:
        print(i,df['segment'].tolist())
# 240 ids are in the control for both 31 and 9, all 40 anyway

In [74]:
df_id_segment_label['segment']=df_id_segment_label['segment'].str.replace("_31_","_40_")
df_id_segment_label['segment']=df_id_segment_label['segment'].str.replace("_9_","_40_")
df_id_segment_label=df_id_segment_label.drop_duplicates()

print(df_id_segment_label.shape,df_id_segment_label['indiv_key'].nunique())
df_id_group_count=df_id_segment_label.groupby("segment")['indiv_key'].count().to_frame().reset_index()
df_id_group_count=df_id_group_count.rename(columns={"indiv_key":"total_ids"})

(284489, 2) 284489


In [68]:
df_both_by_id=pd.merge(df_both_by_id,df_id_segment_label,on="indiv_key",how="left")
df_both_by_id['segment']=df_both_by_id['segment'].fillna("balance")

In [75]:
func_agg={"indiv_key":"count","Ecomm_Trans":"sum","Ecomm_Revenue":"sum",
          "Store_Trans":"sum","Store_Revenue":"sum",
          "Total_Trans":"sum","Total_Revenue":"sum"}

df_output_audience=df_both_by_id.groupby('segment').agg(func_agg).reset_index()
df_output_audience=df_output_audience.rename(columns={"indiv_key":"shoppers"})
df_output_audience=pd.merge(df_id_group_count,df_output_audience,on="segment",how="outer")
df_output_audience=df_output_audience.sort_values("segment")

# Store Report

In [88]:
list_40_test_stores_biqquery=str(list_40_test_stores).replace("[","(").replace("]",")")
list_40_control_stores_biqquery=str(list_40_control_stores).replace("[","(").replace("]",")")
list_80_all_stores_biqquery=str(list_40_test_stores+list_40_control_stores).replace("[","(").replace("]",")")

In [101]:
# Service & Tire & null
print(datetime.datetime.now())
query_both_by_store="""
            select store_id, min(invoice_date) as min_date, max(invoice_date) as max_date, 
            count(distinct indiv_key) as shoppers,
            count(distinct trn_id) as Store_Trans,
            sum(line_item_total_amt) as Store_Revenue
            from mdb.trn_detail
            where store_id in %s and invoice_date between %s and %s
            group by store_id
            """  %(list_80_all_stores_biqquery,str_start_date,str_end_date)
print(query_both_by_id)
df_both_by_store=client.query(query_both_by_store).result().to_dataframe()
print(df_both_by_store.shape,df_both_by_store['store_id'].nunique())
print(datetime.datetime.now())

2020-12-09 00:59:08.636600

            select indiv_key, min(invoice_date) as min_date, max(invoice_date) as max_date, 
            count(distinct(case when store_id='ECOM' then trn_id end)) as Ecomm_Trans,
            sum(case when store_id='ECOM' then line_item_total_amt else 0 end) as Ecomm_Revenue,
            count(distinct(case when store_id!='ECOM' then trn_id end)) as Store_Trans,
            sum(case when store_id!='ECOM' then line_item_total_amt else 0 end) as Store_Revenue,
            count(distinct trn_id) as Total_Trans,
            sum(line_item_total_amt) as Total_Revenue
            from mdb.trn_detail
            where indiv_key is not null and invoice_date between '2020-11-25' and '2020-12-02'
            group by indiv_key
            
(80, 6) 80
2020-12-09 00:59:10.277765


/home/jliang/.local/lib/python3.7/site-packages/google/cloud/bigquery/client.py:441: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  "Cannot create BigQuery Storage client, the dependency "


In [103]:
df_both_by_store['group']=np.where(df_both_by_store['store_id'].isin(list_40_test_stores),"test","control")
df_output_by_store=df_both_by_store.groupby("group")['shoppers','Store_Trans',"Store_Revenue"].sum().reset_index()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


In [104]:
writer=pd.ExcelWriter("./GY_post_campaign_performance_JL_%s.xlsx"%(datetime.datetime.now().date()),engine="xlsxwriter")
df_output_audience.to_excel(writer,"summary_by_audience",index=False)
df_output_by_store.to_excel(writer,"summary_by_store",index=False)
df_both_by_store.to_excel(writer,"output_by_StoreGroup",index=False)
writer.save()


In [100]:
23699.0/23459.0

1.0102306151157339